In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
# from torch.optim.lr_scheduler import StepLR
import mmap
import random
import pickle
import matplotlib.pyplot as plt
import torch.optim as optim
# from torch.optim.lr_scheduler import CosineAnnealingLR

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

batch_size = 126
block_size = 64    
max_iters = 6000  
learning_rate =1.5e-7
eval_iters = 500    
n_embd = 256      
n_head = 12       
n_layer = 12        
dropout = 0.2   



# step_size=1000
# gamma=0.5

# patience = 5  # Number of steps to wait for improvement
# best_val_loss = float('inf')  # Initialize best validation loss
# patience_counter = 0  # Initialize patience counter

cuda


In [2]:
chars = ""
with open('vocab_wiki_cleaned.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(set(text) | {' '})
vocab_size = (len(chars))
print(len(chars))

96


In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [4]:
# memory map for using small snippets of text from a single file of any size
def get_random_chunk(split):
    filename = "train_wiki_cleaned.txt" if split == 'train' else "val_wiki_cleaned.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
            
    return data


def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix]).to(device)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]).to(device)
    return x, y
# try:
#     x, y = get_batch('train')
#     print("Batch retrieved successfully.")
# except Exception as e:
#     print(f"An error occurred: {e}")


In [5]:
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k]= loss.item()
        out[split] =losses.mean()
    return out

In [6]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias = False)
        self.query = nn.Linear(n_embd, head_size, bias = False)
        self.value = nn.Linear(n_embd, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k= self.key(x)
        q= self.query(x)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei= wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei,dim=-1)
        wei = self.dropout(wei)
        v=self.value(x)
        out = wei @ v 
        return out
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim = -1)
        out = self.dropout(self.proj(out))
        return out


class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffws = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self,x):
        y =self.sa(x)
        x =self.ln1(x+y)
        y =self.ffws(x)
        x = self.ln1(x+y)
        return x



class GptLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size , n_embd)
        self.block = nn.Sequential(*[Block(n_embd, n_head = n_head)for _ in range (n_layer)])

        self.ln_f = nn.LayerNorm(n_embd) #final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean = 0.0 , std = 0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean = 0.0 , std = 0.02)
        
    
    def forward(self, index, targets= None):
        B, T = index.shape

        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device= device))
  
        x= tok_emb + pos_emb #b,t,c
        x = self.block(x)
        x= self.ln_f(x)
        logits = self.lm_head(x)


        
        if targets is None:
            loss = None
        else:
            B, T ,C= logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits= logits[:, -1, :]
            probs = F.softmax(logits, dim =-1)
            index_next  = torch.multinomial(probs, num_samples = 1 )
            index = torch.cat((index,index_next), dim = 1)
        return index
model = GptLanguageModel(vocab_size)
# print('loading model parameters...')
# with open('model-wiki-10.pkl', 'rb') as f:
#     checkpoint = pickle.load(f)
#     print("Checkpoint loaded successfully.")
    
#     print("Loading model state dictionary...")
#     model.load_state_dict(checkpoint['model_state_dict'])
#     print("Model state loaded.")
#     optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
#     print("Loading optimizer state dictionary...")
#     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#     print("Optimizer state loaded.")
    
# print("Model and optimizer have been successfully loaded and restored.")

m = model.to(device)

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_iters)

train_losses = []
val_losses = []

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        train_losses.append(losses['train'])
        val_losses.append(losses['val'])
        print(f"step:{iter}, train loss : {losses['train']}, val loss: {losses['val']}")

    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    # scheduler.step()

    # current_lr = optimizer.param_groups[0]['lr']
    # print(f"Current learning rate: {current_lr:.6e}")

print(loss.item())


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss', color='blue')
plt.plot(val_losses, label='Validation Loss', color='orange')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Training and Validation Loss over Iterations')
plt.legend()
plt.grid()
plt.show()

In [13]:
print("Starting to save model and optimizer state...")
with open('model-wiki-10.pkl', 'wb') as f:
    pickle.dump({'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, f)
print('Model and optimizer saved')

Starting to save model and optimizer state...
Model and optimizer saved
